In [1]:
"""
<td>
    <div class="obc-tmpl-character__voice-item obc-tmpl-character__play-voice">
        <div class="obc-tmpl-character__voice-btn">
        </div> 
        <span class="obc-tmpl-character__voice-content"> 	
            [text] 	 
        </span>
    </div>
 </td>
"""

'\n<td>\n    <div class="obc-tmpl-character__voice-item obc-tmpl-character__play-voice">\n        <div class="obc-tmpl-character__voice-btn">\n        </div> \n        <span class="obc-tmpl-character__voice-content"> \t\n            [text] \t \n        </span>\n    </div>\n </td>\n'

In [2]:
URL = "https://bbs.mihoyo.com/ys/obc/content/6938/detail"
marks=["C","E","J","K"]


In [3]:
import time
from seleniumwire import webdriver  # pip install selenium-wire
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

# Configuration constants
GECKODRIVER_PATH = "/home/co/firefox/geckodriver"
FIREFOX_BINARY = "/home/co/firefox/firefox/firefox"
OUTPUT_DIR = "/tmp/audio"

# Define a request interceptor to add "Cache-Control: no-cache" to every request.
def interceptor(request):
    request.headers['Cache-Control'] = 'no-cache'

options = Options()
options.binary_location = "/home/co/firefox/firefox/firefox"
options.add_argument('--headless')
options.add_argument("--width=1920")
options.add_argument("--height=1080")

# Initialize selenium-wire Firefox driver with our interceptor.
driver = webdriver.Firefox(
    service=Service("/home/co/firefox/geckodriver"),
    options=options,
)
driver.request_interceptor = interceptor

In [4]:
# Navigate to the target URL and wait until at least one container is present.
driver.get(URL)
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "div.obc-tmpl-character__voice-item"))
)

# Locate all container elements that include both the text and the voice button.
containers = driver.find_elements(
    By.CSS_SELECTOR,
    "div.obc-tmpl-character__voice-item.obc-tmpl-character__play-voice"
)

results = []
failed=0

for index, container in enumerate(containers):
    # Extract text from the container.
    try:
        text_el = container.find_element(By.CSS_SELECTOR, "span.obc-tmpl-character__voice-content")
        text = text_el.text.strip()
    except Exception:
        text = ""
    
    # Locate the voice button.
    try:
        button = container.find_element(By.CSS_SELECTOR, "div.obc-tmpl-character__voice-btn")
    except Exception:
        print(f"Button not found in container {index}")
        results.append((text, None))
        continue

    # Scroll the container into view.
    driver.execute_script("arguments[0].scrollIntoView(true);", container)
    
    mp3_url = ""
    max_attempts = 10  # Total wait: ~5 seconds (10*0.5)
    attempt = 0

    # Clear previous network requests.
    del driver.requests

    while attempt < max_attempts and not mp3_url:
        try:
            ActionChains(driver).click(button).perform()
        except Exception as e:
            print(f"Error clicking button in container {index}: {e}")
        # Wait a bit for network requests to occur.
        time.sleep(0.5)
        # Check driver.requests for a GET request with URL ending in .mp3.
        for request in driver.requests:
            if request.method == "GET" and request.url.lower().endswith(".mp3"):
                mp3_url = request.url
                print(f"Captured mp3 for container {index}: {mp3_url}")
                break
        attempt += 1
        driver.requests.clear()

    if not mp3_url:
        print(f"No mp3 request captured for container {index}")
        failed+=1
    results.append((text, mp3_url))

driver.quit()
print(f"# of FAILED: {failed}")

Captured mp3 for container 0: https://act-upload.mihoyo.com/ys-obc/2023/08/23/16576950/019c879112383432d647c1520a7f12e6_8338346760704900577.mp3
Captured mp3 for container 1: https://act-upload.mihoyo.com/ys-obc/2023/08/23/16576950/e2ff09436bc7317853f0981cced9edc8_465303680260218006.mp3
Captured mp3 for container 2: https://act-upload.mihoyo.com/ys-obc/2023/08/23/16576950/dab91e5dc0f60c232917684659681941_4316447749029330386.mp3
Captured mp3 for container 3: https://act-upload.mihoyo.com/ys-obc/2023/08/23/16576950/d97f20da813c0a985bcd36b3708e053b_1381326163251514108.mp3
Captured mp3 for container 4: https://act-upload.mihoyo.com/ys-obc/2023/08/23/16576950/51a729d322a57457b69c3b4b796afa65_5657519325105310090.mp3
Captured mp3 for container 5: https://act-upload.mihoyo.com/ys-obc/2023/08/23/16576950/1ffa3c5cd9766d57d125130bd1061f34_520520351788677303.mp3
Captured mp3 for container 6: https://act-upload.mihoyo.com/ys-obc/2023/08/23/16576950/bd6dc7017049fa47dd9b3eafffc28987_170526295407905614

In [5]:
len(list(set([y for x,y in results])))==len(results)

True

In [6]:
import os
import requests
quarter = len(results) // 4
# Headers from network inspection
HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:136.0) Gecko/20100101 Firefox/136.0",
    "Referer": "https://bbs.mihoyo.com/",
    "Accept": "audio/webm,audio/ogg,audio/wav,audio/*;q=0.9,application/ogg;q=0.7,video/*;q=0.6,*/*;q=0.5",
    "DNT": "1",
    "Sec-Fetch-Dest": "audio",
    "Sec-Fetch-Mode": "no-cors",
    "Sec-Fetch-Site": "same-site"
}
person_id=URL.split('/')[-2]
len(results),quarter,person_id

(284, 71, '6938')

In [7]:
for i, (text, url) in enumerate(results):
    if url:
        q = i // quarter
        response = requests.get(url, headers=HEADERS, stream=True)
        response.raise_for_status()
        dir=os.path.join(OUTPUT_DIR,person_id,marks[q])
        os.makedirs(dir, exist_ok=True)
        with open(os.path.join(dir,f"{i-q*quarter}.mp3"),"wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:  # Filter out keep-alive chunks
                    f.write(chunk)



In [8]:
dir=os.path.join(OUTPUT_DIR,person_id)
os.makedirs(dir, exist_ok=True)
with open(os.path.join(dir,"index.csv"),"w") as f:
    i=0
    while i<quarter:
        f.write(f"{i},{results[i][0]}\n")
        i+=1


In [10]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import os
import librosa


device = "cpu"#"cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float32#torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"  # Note: Corrected model ID from original

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps="word",  # Changed from True to "word" for word-level timestamps
    generate_kwargs={
        "task": "transcribe",
        "language": "english",  # Specify your audio's language
    },
)

Device set to use cpu


In [11]:
mp3_dir=os.path.join(dir,'E')

In [12]:

results = []
for filename in os.listdir(mp3_dir):
    if filename.lower().endswith('.mp3'):
        file_path = os.path.join(mp3_dir, filename)
        
        # Load audio with librosa (properly resampled)
        audio, sr = librosa.load(file_path, sr=16000, mono=True)
        
        # Process with timestamp generation
        result = pipe(
            {"array": audio, "sampling_rate": sr},
            return_timestamps="word",  # Ensure timestamp generation
        )
        
        results.append(
           (int(os.path.splitext(filename)[0]),result["text"])
        )

/home/co/miniconda3/envs/3.12/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
WhisperModel is using WhisperSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `layer_head_mask` not None. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProc

In [ ]:
results=sorted(results)
results

[(0,
  ' Yaya Publishing House has released numerous works that lay out military strategy in a simple and pragmatic way. Those are my favorite.'),
 (1,
  ' Respect must be given to the will of every creature. Each fish in the ocean swims in its own direction.'),
 (2, ' To survive hardship, you must prepare for hardship.'),
 (3,
  " I am Sangonomiya Kokomi, the Divine Priestess of Watatsumi Island. My journey with you will be an opportunity to unwind. I mean, to survey beyond our borders. Don't worry, I have left behind ample directives in my absence and have also completed the island's projected development plans. Departing for a brief period will be of little consequence."),
 (4,
  ' Marching in the rain conceals the sound of our arrival, but is also a test of stamina.'),
 (5, ' The snow... Have you ever seen such beauty?'),
 (6, ' Such fortunate weather. Care for a walk?'),
 (7,
  ' Hmm. High winds. An incendiary attack would be particularly effective.'),
 (8,
  " Good morning. I've 

In [ ]:
with open(os.path.join(mp3_dir,"index.csv"),"w") as f:
    for idx,text in results:
        f.write(f"{idx},{text}\n")
